In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from konlpy.tag import Okt

import os

from keybert import KeyBERT
import tqdm

c:\ProgramData\Anaconda3\envs\DINS\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
data = pd.read_csv('./Data/SBS.csv')

data.head(1)

,title,headline,date,link,content,category,site
0,"이란 남부 원유시설서 화재…업체 대표 ""정체불명 세력이 방화""",이란 남부 원유시설에서 불이 나 한때 생산이 중단됐다고 현지 언론이 13일 보도했습...,2022.09.13,https://news.sbs.co.kr//news/endPage.do?news_i...,이란 남부 원유시설에서 불이 나 한때 생산이 중단됐다고 현지 언론이 13일(현지...,국제,sbs


In [45]:
# date 변경, link col_name 변경 및 저장

# data.rename(columns={'news_link' : 'link'}, inplace=True)

# data.date = data.date.str.split(' ').str[0]

# data.head(1)

# data.to_csv('./Data/SBS.csv', index=False)

In [26]:
# okt = Okt()

# contents_nouns = []

In [6]:
# contents_nouns.append(p.map(get_nouns, contents))

In [4]:
# for text in contents:
#     nouns = okt.nouns(text)
    
#     text_nouns = ''
#     for noun in nouns:
#         text_nouns += noun + ' '
    
#     contents_nouns.append(text_nouns)
#     print(text_nouns)

In [22]:
# contents = data.iloc[79573:].content

# len(contents)

252

In [4]:
# data_sep = data.iloc[78237:]

# len(data_sep)

1588

In [8]:
dates = data.date.unique()

len(dates)
# 2022.09.13 ~ 2022.10.30

48

In [9]:
categories = data.category.unique()

len(categories)
# '사회', '국제', '스포츠', '생활·문화', '경제', '정치', '연예'

7

In [10]:
data_date = []

for date in dates:
    data_date.append(data.groupby('date').get_group(date))

In [11]:
datas = []


for data in data_date:
    data_category = []
    for category in categories:
        try:
            data_category.append(data.groupby('category').get_group(category).content)
        except (KeyError):
            pass
    
    datas.append(data_category)

In [12]:
okt = Okt()

contents_nouns = []

In [13]:
datas_nouns = []

for data_date in datas:
    contents_nouns = []
    
    for contents in data_date:
        content_nouns = ''
        
        for text in contents:
            text_nouns = okt.nouns(text)
        
            for noun in text_nouns:
                content_nouns += noun + ' '
            
        contents_nouns.append(content_nouns)
        
    datas_nouns.append(contents_nouns)

In [25]:
# content_nouns = ''

# for text in contents:
#     text_nouns = okt.nouns(text)
    
#     for noun in text_nouns:
#         content_nouns += noun + ' '
    

# contents_nouns.append(content_nouns)

In [14]:
model = KeyBERT('distilbert-base-nli-mean-tokens')

In [15]:
datas_keywords = []

for data_date in datas_nouns:
    contents_keywords = []
    
    for content_nouns in data_date:
        contents_keywords.append(model.extract_keywords(content_nouns, keyphrase_ngram_range=(1,1), top_n=20, use_mmr=True))
    
    datas_keywords.append(contents_keywords)

In [16]:
result_keywords = []

date_idx = 0
category_idx = 0

for data_keywords in datas_keywords:
    for keywords in data_keywords:
        for keyword in keywords:
            result_keywords.append((dates[date_idx], categories[category_idx], ) + keyword) # (date, category, keyword, importance)
    
        category_idx += 1
            
    date_idx += 1
    category_idx = 0

In [17]:
result = pd.DataFrame(result_keywords, columns=['date', 'category', 'keyword', 'importance'])

result

,date,category,keyword,importance
0,2022.09.13,국제,마이크로소프트,0.8253
1,2022.09.13,국제,원삼국시대,0.8196
2,2022.09.13,국제,중화인민공화국,0.8195
3,2022.09.13,국제,산업혁명,0.8083
4,2022.09.13,국제,다큐멘터리,0.8062
...,...,...,...,...
6695,2022.10.30,연예,축구선수,0.7161
6696,2022.10.30,연예,여기저기,0.7111
6697,2022.10.30,연예,글로벌,0.6980
6698,2022.10.30,연예,대부분,0.6929


In [18]:
result.to_csv('./Data/keywords.csv', index=False)

In [19]:
import pymysql

In [20]:
keywords = pd.read_csv('./Data/keywords.csv')

keywords

,date,category,keyword,importance
0,2022.09.13,국제,마이크로소프트,0.8253
1,2022.09.13,국제,원삼국시대,0.8196
2,2022.09.13,국제,중화인민공화국,0.8195
3,2022.09.13,국제,산업혁명,0.8083
4,2022.09.13,국제,다큐멘터리,0.8062
...,...,...,...,...
6695,2022.10.30,연예,축구선수,0.7161
6696,2022.10.30,연예,여기저기,0.7111
6697,2022.10.30,연예,글로벌,0.6980
6698,2022.10.30,연예,대부분,0.6929


In [22]:
sql_insert = 'INSERT INTO keywords (date, category, keyword, importance) VALUES (%s, %s, %s, %s)'

In [23]:
conn = pymysql.connect(host='localhost', user='root', password='1234', db='dins', charset='utf8')
cursor = conn.cursor()


for index, row in keywords.iterrows():
    cursor.execute(sql_insert, (row.date, row.category, row.keyword, row.importance))

conn.commit()
conn.close()

In [106]:
# keywords = model.extract_keywords(contents_nouns, keyphrase_ngram_range=(1,1), top_n=15, use_mmr=True)

# keywords